In [1]:
import os

In [2]:
os.getcwd()

'c:\\Users\\m3irs\\OneDrive\\Desktop\\2021DataScience\\2023Datascience\\Project\\wine_quality_project\\research'

In [3]:
os.chdir('../')

In [4]:
os.getcwd()

'c:\\Users\\m3irs\\OneDrive\\Desktop\\2021DataScience\\2023Datascience\\Project\\wine_quality_project'

In [5]:
from dataclasses import dataclass
from pathlib import Path



@dataclass
class ModelTrainerConfig:
    root_dir:Path
    train_data_path:Path
    test_data_path:Path
    model_name:str
    alpha:float
    l1_ratio:float
    target_column:str
    

In [7]:
# Define the Cconfiguration Manager

from src.mlproject.constants import constant
from src.mlproject.utils.common import read_yaml,create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = constant.CONFIG_FILE_PATH,
        params_filepath = constant.PARAMS_FILE_PATH,
        schema_filepath = constant.SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
        
        
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN
        
        create_directories([config.root_dir])
        
        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            alpha=params.alpha,
            l1_ratio=params.l1_ratio,
            target_column=schema.name
            
        )
        
        
        return model_trainer_config
        
        
        

In [8]:
read_yaml(constant.CONFIG_FILE_PATH)

config\config.yaml


ConfigBox({'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_url': 'https://github.com/Irshad-ml/Dataset-Collection/raw/main/winequality-data.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}, 'data_validation': {'root_dir': 'artifacts/data_validation', 'unzip_data_dir': 'artifacts/data_ingestion/winequality-red.csv', 'STATUS_FILE': 'artifacts/data_validation/status.txt'}, 'data_transformation': {'root_dir': 'artifacts/data_transformation', 'data_path': 'artifacts/data_ingestion/winequality-red.csv'}, 'model_trainer': {'root_dir': 'artifacts/model_trainer', 'train_data_path': 'artifacts/data_transformation/train.csv', 'test_data_path': 'artifacts/data_transformation/test.csv', 'model_name': 'model.joblib'}})

In [9]:
#Update the component
import pandas as pd
import os
from src.mlproject.logger import logging
from sklearn.linear_model import ElasticNet
import joblib


class ModelTrainer:
    def __init__(self,config=ModelTrainerConfig):
        self.config=config
        
        
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)
        
        train_x = train_data.drop([self.config.target_column],axis=1)
        test_x = test_data.drop([self.config.target_column],axis=1)
        
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]
        
        # train the model
        lr = ElasticNet(alpha=self.config.alpha,l1_ratio=self.config.l1_ratio,random_state=42)
        lr.fit(train_x,train_y)
        
        #save the model
        joblib.dump(lr,os.path.join(self.config.root_dir,self.config.model_name))
        





In [10]:
try:
    config=ConfigurationManager()
    model_trainer_config=config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()
    
    
except Exception as e:
    raise e

config\config.yaml
params.yaml
schema.yaml
